# F1 Strategy Assistant – MVP

This notebook demonstrates a GenAI assistant that helps F1 strategists decide when to pit, using real-time race data.

---

In Formula 1, deciding the optimal time for a pit stop is a complex and high-stakes challenge. Strategists must balance a wide range of factors — tyre wear, compound choice, weather, track conditions, and more — all while reacting to real-time developments like safety cars or opponent decisions. These decisions must be made quickly and under pressure, leaving room for human error.

This project explores how a GenAI-powered assistant can support F1 strategists by:

- Continuously analyzing race context based on numerical inputs.
- Providing timely pit recommendations grounded in data.
- Reducing cognitive load by automating data-driven decision-making.
- Offering recalculated guidance when unexpected changes occur (e.g., strategy deviations, weather shifts).

By offloading number-driven tasks to the AI, the human strategist can focus on elements influenced by competitor behavior and race psychology — areas where human judgment excels.


## Setup for Data Engineering and Model Training

To begin, we install the `fastf1` library. This API gives us access to lap-by-lap telemetry and session data, which is essential for building a model that understands race dynamics.

In [74]:
!pip install fastf1

In [75]:
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import numpy as np
import fastf1 as ff1
import fastf1.plotting

# Data Collection 

We define a function `gpData(year, country, session)` that:

- Loads session data for a given race.
- Extracts lap-level data using FastF1.
- Computes several additional features:
  - **DeltaTime:** Lap-to-lap performance variation for each driver.
  - **GapAhead:** Average time gap to the car ahead on each lap.
  - **PosToLose:** How many positions a driver would likely lose if they pitted at that moment, based on the pit lane time loss and gaps ahead.
  - **RemainingLaps:** Number of laps left in the race.
- Builds the target series `pitting`, which indicates whether a car pitted on a given lap.

The function returns:
- A processed `DataFrame` with race features.
- A binary `Series` as the prediction target.


In [76]:
def gpData(year, country, session):
    """
    Parameters:
        year (int): The year of the race (e.g., 2023, 2024).
        country (str): The country where the race is held (e.g., 'Saudi Arabia').
        session (str): The session of the race (e.g., 'r' for race, 'q' for qualifying).

    Returns:
        race (DataFrame): A DataFrame containing lap-level data and derived features.
        pitting (Series): A series indicating whether a pit stop occurred (1 for pit, 0 for no pit).
    """

    # Load the specified session using the FastF1 API
    session = ff1.get_session(year, country, session)
    session.load()

    # Extract lap data and sort by lap number and position
    laps = session.laps
    laps.sort_values(by=['LapNumber', 'Position'], inplace=True)
    laps.reset_index(drop=True, inplace=True)

    # Get weather data for the session
    weather = laps.get_weather_data()

    # Create the race DataFrame with relevant lap-level data
    race = pd.DataFrame({
        'LapNumber': laps['LapNumber'].values,
        'Position': laps['Position'].values,
        'Compound': laps['Compound'].values,
        'TyreLife': laps['TyreLife'].values,
        'TrackStatus': laps['TrackStatus'].map(lambda x: 1 if x == 4 else (2 if x == 6 else 0)),
        'TrackTemp': weather['TrackTemp'].values,
        'Rainfall': weather['Rainfall'].values.astype(int)
    })


    # Calculate the delta time between consecutive laps for each driver
    lap_time_delta = pd.Series()

    for i, driver in laps.groupby('Driver'):
        delta =[]
        delta = driver['LapTime'].dt.total_seconds().diff()
        delta.fillna(0, inplace=True)
        lap_time_delta = pd.concat([lap_time_delta, delta])

    race['DeltaTime'] = lap_time_delta


    # Calculate average distance to the driver ahead over a lap in seconds
    gap = []
    for i, l in laps.iterlaps():
        dist = l.get_telemetry().DistanceToDriverAhead
        speed = l.get_telemetry().Speed*3.6

        gap.append((dist/speed).mean())

    race['GapAhead'] = gap
    race['GapAhead'].replace(np.inf,np.nan, inplace=True)
    race['GapAhead'].fillna(0, inplace=True)

    
    # Add country of the race
    race['Country'] = session.event.Country

    
    # Calculate the number of positions that would be lost if the driver pitted in that lap
    lost_pos = []

    for i,group in race.groupby('LapNumber'):
        k = 0
        for j in group.index:
            if group.loc[j,'TrackStatus'] == 0:
                pit_loss = pit_stop_cost[group.loc[j,'Country']]
                
            else :
                pit_loss = pit_stop_cost_sc[group.loc[j,'Country']]

            counter = 1
            loss = 0
            while (loss < pit_loss) & ((k+counter) < group.index.size):
                loss += group.loc[j+counter,'GapAhead']
                counter += 1

            lost_pos.append(counter)
            k += 1

    race['PosToLose'] = lost_pos

    
    race['TotalLaps'] = laps['LapNumber'].max()
    race['RemainingLaps'] = race['TotalLaps'] - race['LapNumber']

    
    laps.sort_values(by=['Driver', 'LapNumber'], inplace=True)
    
   # Define the condition when a car is making a pit stop
    pit_stop_condition = (
    (laps['PitInTime'].notna()) &
    (laps['TyreLife'] >= laps['TyreLife'].shift(-1)) &
    (laps['PitOutTime'].shift(-1).notna())
    )

    # Create a target series for pit stop (1 for pit, 0 for no pit)
    pitting = pd.Series(np.where(pit_stop_condition, 1, 0))
    pitting.index = laps.index
    pitting.sort_index(inplace=True)

    return race, pitting

To estimate potential position loss due to pitting, we define dictionaries for pit stop loss in seconds for both normal conditions and for safety car or virtual safety car conditions:

In [77]:
pit_stop_cost={
    'Saudi Arabia': 0.2
}

pit_stop_cost_sc={
    'Saudi Arabia': 0.11

}

To ensure quick iteration and testing, our final dataset will consist of data from a single circuit across two different seasons. This decision was made because processing a single race can take over 30 minutes. By focusing on just one circuit, we maintain a consistent environment while still capturing enough variability for meaningful learning and experimentation.

We chose the 2023 and 2024 seasons of the Saudi Arabian Grand Prix, as it is a shorter circuit. Additionally, it is the next race in the 2025 season, which will provide an opportunity to test the agent live.

In [78]:
saudi23, saudi23_target = gpData(2023, 'Saudi Arabia', 'r')
saudi24, saudi24_target = gpData(2024, 'Saudi Arabia', 'r')

## Data engineering

We prepare the data for the model by combining race data from two different years, cleaning up missing values, encoding categorical variables, and dropping unnecessary columns. This ensures the data is ready for analysis and modeling.

In [79]:
# Combine lap data from the 2 races.
lapData = pd.concat([saudi23, saudi24], ignore_index=True)
target = pd.concat([saudi23_target, saudi24_target], ignore_index=True)

In [80]:
# Drop rows with missing "Position" values.
rows_to_drop = lapData[lapData["Position"].isna()].index
lapData = lapData.drop(rows_to_drop)
target = target.drop(rows_to_drop)

In [81]:
# Encode compound types
lapData['Compound'] = lapData['Compound'].astype('category').cat.codes

# Convert relevant columns into categorical data types.
lapData['Position'] = lapData['Position'].astype('category')
lapData['Compound'] = lapData['Compound'].astype('category')
lapData['Rainfall'] = lapData['Rainfall'].astype('category')

lapData = lapData.drop(columns=['Country'])

# Model Training and Evaluation

We train a Random Forest Classifier to predict the target variable based on the prepared data.

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier(random_state=42, max_depth = 10, min_samples_split = 2, n_estimators = 50, class_weight='balanced')

# Split data into training and test sets (80/20 split).
lapData_train, lapData_test, target_train, target_test = train_test_split(lapData, target, test_size=0.2, random_state=42)

# Fit model
model.fit(lapData_train, target_train)

# Evaluate the model
predictions = model.predict(lapData_test)

# Classification report 
print("\nClassification Report:\n", classification_report(target_test, predictions))


Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       361
           1       0.71      0.62      0.67         8

    accuracy                           0.99       369
   macro avg       0.85      0.81      0.83       369
weighted avg       0.99      0.99      0.99       369



# Building the Strategy Assistant

## Get set up

Start by installing and importing the LangGraph SDK and LangChain support for the Gemini API.

In [83]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

## Set up your API key

In [84]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## Define core instructions

In [85]:
# Standard library
import json
import atexit
from typing import Annotated, Literal
from typing_extensions import TypedDict
from collections import OrderedDict

# Display utilities
from IPython.display import Image, display

# LangGraph core
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.tools import tool

# LangChain Google Generative AI
from langchain_google_genai import ChatGoogleGenerativeAI
  

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

## Define Race State and Assistant Instructions
Set up the race state structure and system prompt that guides how the assistant interacts during the race. Includes examples of expected behavior and tool usage.

In [86]:

class RaceState(TypedDict):
    """State representing the human startegies's pit stop conversation."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The `add_messages` annotation indicates to LangGraph
    # that state is updated by appending returned messages, not replacing
    # them.
    messages: Annotated[list, add_messages]

    # Number of laps in the race
    total_lap_number: float

    # The state of the track
    # {'Green', 'Safety Car', 'Virtual Safety Car'}
    track_status: float
    
    # Weather data
    # [track_temp, rainfall]
    weather: list[float]

    # Log of lap information and related pit decisions
    lap: list[dict]

    # Flag indicating that the race is over.
    finished: bool


# System prompt guiding the assistant’s behavior during the race
STRATEGYASSISTAN_SYSINT = (
    "system", 

    """
    You are an AI race strategist assistant helping a human strategist during a Grand Prix. 
    Your task is to decide whether the car should pit for new tyres based on live race data.  
    
    A human strategist will give you the necessary lap and weather information throughout the race. 
    You must tell them if they should BOX, BOX or STAY OUT this lap, and provide a confidence percentage and a brief explanation of the decision — but nothing more.
    
    ---
    
    Tools & Instructions
    
    - At the start of the race, the human will give you:
        - Total number of laps → store this using `store_total_lap_number`
        - Track status → store using `store_track_status`  
        - weather: track temperature and if there is rain or not → store and update using `update_weather`
    
    - Throughout the race:
        - Make sure you have the the waethr condition, track status and total number of laps before calling `pit_check`
        - You can update the track temperature and if it reains or not using `update_weather`
        - When you get lap information use `pit_check` to get decision if to pit or not
        - Retrieve total number of laps using `get_total_lap_number`
        - Retrieve track status using `get_track_status`
        - Retrieve weather weather information using `get_weather`
      
    
    If a required tool has not yet been implemented, you are allowed to break the fourth wall and tell the human strategist:  
    "This tool is not yet implemented."

    
    Here are four examples demonstrating the expected input, model output, and reply style:
    
    Example 1 – Pit under normal conditions:  

    Human input:
    We’re on lap 21, currently P3, running 20-lap-old Mediums. 
    Our lap delta is -0.13, the gap to the car ahead is 5.97 seconds, and if we box now, we’ll lose 7 positions.

    Respons:  
    Pit, confidence 76%. 
    Reason: BOX, BOX. \n Tyres are reaching the end of their performance window. Pitting now helps minimize position loss before others stop.
    
    ---
    
    Example 2 – Pit in wet conditions:  
    Human input:
    Lap 44. P1. 10-lap Hards. Rain started. Delta +1.2. Gap 0. Lose 3 if we pit. 42 to go.
    
    Respons:  
    Pit, confidence 91%.
    Reason: BOX, BOX. \n Rain is affecting grip, and loss is acceptable given 42 laps remaining.
    
    ---
    
    Example 3 – Stay out under Safety Car:  
    Human input:
    Lap 3, P5. Safety Car. Mediums, 2 laps. Gap 1.3, delta -0.1. Pit drops us 5. 55 left.
    
    Reply:  
    Stay out, confidence 80%. 
    Reason: STAY OUT. \n It’s early in the race and tyres are fresh. Pitting now would lose positions without strategic advantage.
    
    ---
    
    Example 4 – Stay out late in race: 
    Human input:
    We're on lap 46, running P6. Tyres are 20-lap-old hards, still holding up. 
    Gap to car ahead is 2.2, delta's -0.3. If we pit, we lose 2 spots. Only 7 laps left.
    
    Reply:  
    Stay out, confidence 84%. 
    Reason: STAY OUT. \n Tyres are holding up and pitting now would cost positions without meaningful advantage.

    """,

)

# Message with which the system opens the conversation.
WELCOME_MSG = "Hey, I am your Strategy Assistent, and I will be here to help during the Saudi Arabian GP. \n Please give me the number of laps in this race."


### Define tools

These tools let the assistant store and retrieve race data (like total laps, track status, weather) and use it to decide if the car should pit.

In [87]:
@tool
def store_total_lap_number(total_lap_number: float) -> str:
    """Stors the total number of laps in the race
    
    Returns:
        Confirmation message 
    """

@tool
def get_total_lap_number() -> str:
    """Returns the total number of laps in the race"""

@tool
def store_track_status(track_status: str) -> str:
    """Store the current track status (Green, Safety Car, Virtual Safety Car)   
    
    Returns:
        Confirmation message 
    """

@tool
def get_track_status() -> str:
    """Return the current track status"""

@tool
def update_weather(track_temp: float, rainfall: float) -> str:
    """Update the current weather conditions

    Returns:
        Confirmation message.
    """
    
@tool
def get_weather() -> str:
    """Returns the latest weather conditions"""


@tool
def pit_check(lap_number: float, position: float, tyre_life: float, compound: str, delta_time: float, gap_ahead: float, pos_to_lose: float) -> [float]:
    """Pass lap and race conditions to the model and get a pit decision

    Returns:
        Model prediction and confidence

    """

### Define control flow and race logic

This section connects the model, tools, and human input. 
It routes messages between the user, the AI, and the tools. 
It also defines how race data is processed and stored.


In [88]:
def race_node(state: RaceState) -> RaceState:
    """Processes tool calls and updates race state."""
    
    tool_msg = state.get("messages", [])[-1]
    total_lap_number = state.get("total_lap_number", [])
    track_status = state.get("track_status", [])
    weather = state.get("weather", [])
    lap = state.get("lap",[])
    outbound_msgs = []
    lap_log = state.get("log",[])
    race_finished = False

    for tool_call in tool_msg.tool_calls:
        if tool_call["name"] == "store_total_lap_number":
            total_lap_number = f'{tool_call["args"]["total_lap_number"]}'
            response = '\n'.join(total_lap_number)
            
        elif tool_call["name"] =="get_total_lap_number":
            response = "\n".join(total_lap_number) if total_lap_number else "No total lap number"

        elif tool_call["name"] == "store_track_status":
            track_status = f'{tool_call["args"]["track_status"]}'
            response = '\n'.join(track_status)
            
        elif tool_call["name"] =="get_track_status":
            response = "\n".join(track_status) if track_status else "No track infomration"

        elif tool_call["name"] == "update_weather":
            weather = (f'{tool_call["args"]["track_temp"]}, {tool_call["args"]["rainfall"]}')
            response = "\n".join(weather)

            
        elif tool_call["name"] == "get_weather":
            response = "\n".join(weather) if weather else "No weather data"
            
        
        elif tool_call["name"] == "pit_check":
            # Extract inputs
            lap_number = int(tool_call["args"]["lap_number"])
            position = tool_call["args"]["position"]
            tyre_life = float(tool_call["args"]["tyre_life"])
            delta_time = tool_call["args"]["delta_time"]
            gap_ahead = tool_call["args"]["gap_ahead"]
            pos_to_lose = tool_call["args"]["pos_to_lose"]
            total_laps = float(total_lap_number)
            remaining_laps = total_laps - lap_number

            # Convert compound to numeric value
            compound_str = tool_call["args"]["compound"]
            compound_value = (
                2.0 if compound_str in ["Medium", "Mediums"] else
                3.0 if compound_str in ["Soft", "Softs"] else
                1.0 if compound_str in ["Intermediates", "Inters"] else
                4.0 if compound_str in ["Wet", "Wets"] else
                0.0
            )

            # Convert track status to numeric value
            track_status_value = (
                1.0 if track_status == "Safety Car" else
                2.0 if track_status == "Virtual Safety Car" else
                0.0
            )

            # Parse weather info
            weather_data = weather.split(",")
            track_temp = float(weather_data[0].strip())
            rainfall = float(weather_data[1].strip())
            
            # Create lap feature vector
            lap_data = {
                "LapNumber": lap_number,
                "Position": position,
                "Compound": compound_value,
                "TyreLife": tyre_life,
                "TrackStatus": track_status_value,
                "TrackTemp": track_temp,
                "Rainfall": rainfall,
                "DeltaTime": delta_time,
                "GapAhead": gap_ahead,
                "PosToLose": pos_to_lose,
                "TotalLaps": total_laps,
                "RemainingLaps": remaining_laps
            }
            
            # Run prediction
            model_prediction_proba = model.predict_proba(pd.DataFrame([lap_data]))

            if model_prediction_proba[0,1] >= 75:
                lap_data["Decision"] = "BOX BOX"
                lap_data["Confidence"] = float(model_prediction_proba[0][1])
                response = "\n".join(f'Stay out, confidence {model_prediction_proba[0][1]}')
            else:
                lap_data["Decision"] = "STAY OUT"
                lap_data["Confidence"] = float(model_prediction_proba[0][0])
                response = "\n".join(f'Stay out, confidence {model_prediction_proba[0][0]}')

            
            lap.append(OrderedDict(lap_data))
            
        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        # Record tool call response
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {
        "messages": outbound_msgs,
        "track_status": track_status,
        "weather": weather,
        "total_lap_number": total_lap_number,
        "lap": lap,
        "finished": race_finished
        }


def human_node(state: RaceState) -> RaceState:
    """Displays model output and waits for human input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # End session if user types a quit keyword
    if user_input in {"q", "quit", "exit"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}
    

def maybe_exit_human_node(state: RaceState) -> Literal["chatbot", "__end__"]:
    """Route to chatbot or end based on user input."""
    return END if state.get("finished", False) else "chatbot"


def chatbot(state: RaceState) -> RaceState:
    """Main chatbot logic including tool calls."""
    defaults = {"race_info": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([STRATEGYASSISTAN_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {"messages": [new_output]}


def maybe_route_to_tools(state: RaceState) -> str:
    """Route to the correct next node depending on the message type."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        return END

    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        return "racing"
    else:
        return "human"


# Define the app graph
graph_builder = StateGraph(RaceState)

# Tool list
race_tools = [
    store_total_lap_number,
    get_total_lap_number,
    store_track_status,
    get_track_status,
    update_weather,
    get_weather,
    pit_check
]

# Bind tools to the model
llm_with_tools = llm.bind_tools(race_tools)

# Add nodes
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("human", human_node)
graph_builder.add_node("racing", race_node)

# Set flow logic
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
graph_builder.add_conditional_edges("human", maybe_exit_human_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("racing", "chatbot")

# Final app
chat_with_human= graph_builder.compile()

## Running the Strategy Assistent

To run the agent, you need to uncomment the `.invoke(...)` line in the code. 
This line has been commented out in the notebook to make it self-contained for initial setup. Once you’re ready to run the agent, simply remove the comment to enable the execution of the model.

Additionally, a few examples of human input have been provided in the comments following that line. These examples simulate real-world race data and help evaluate the agent’s performance during testing.

Disclaimer:
The model’s performance on race data from other seasons may be suboptimal, likely due to missing contextual features that influence real-world strategy decisions. However, for the purpose of this project—demonstrating the capabilities of a GenAI-powered agent—the current performance is sufficient. Improving the model's accuracy by incorporating additional features is part of planned future work.

In [89]:
from pprint import pprint

config = {"recursion_limit": 100}

# Remember that this will loop forever, unless you input "q", "quit" or "exit"


# Uncomment this line to execute the graph:
# state = chat_with_human.invoke({"messages": []}, config)


# example 1:
# Model: Hey, I am your Strategy Assistent, and I will be here to help during the Saudi Arabian GP. Please give me the number of laps in this race.

# User:  50 laps
# User:  safety care is out, track temp 34, no rain
# Used: we are p1 on lap 3, 2 lap old soft tyres, gap to driver ahead 0.8, delta -1.6, losing 13 position if we pit
# Expected answer: STAY OUT


# example 2:
# User:  race has 50 laps
# User:  track is green, track temp 31.8, no rain
# User: lap 16 p7. 15 lap old hards delta 2.358, gap ahread 0.336196 would lose 2
# Expected answer: BOX BOX


# example 3:
# User:  50 laps
# User:  track is green, track temp 28.3, no rain
# User:  lap 15, position 6, 14 laps old mediums, delta -0.19, gap ahead 0.97, losing 4 positions
# Expected answer: STAY OUT

# User:  lap 16, position 5, 15 laps old mediums, delta -0.15, gap ahead 1.1, losing 5 positions
# Expected answer: Either decision wuld be acceptable

# User:  lpa 17 p 2 16 used medium, delta -0.10, gap ahead 1.6, losing 4 positions
# Expected answer: BOX BOX




### 💾 Saving the Agent Decision Log as a JSON File

The `lap_log` stores all relevant race information and the agent’s decisions across multiple laps.  
Saving it as a `.json` file allows us to easily review the model's performance after a race simulation.  
This helps us better understand the models’s behavior, evaluate its decision-making, and identify areas for future improvement.


In [90]:
with open("lapLog.json", "w") as f:
    json.dump(state['lap'], f, indent=4)

# Summary

This project demonstrates the potential of GenAI to assist Formula 1 strategists in making critical pit stop decisions during live races. By leveraging race data such as lap number, tyre conditions, track status, and more, the GenAI assistant offers timely, data-driven recommendations on whether to pit or stay out.

## GenAI Capabilities Used:

- **Agents**: The core of this project is built around a GenAI-powered agent that functions as a "Pit Wall Assistant." This agent processes real-time race data and outputs recommendations based on learned patterns.

- **Few-Shot Prompting**: Through the use of few-shot prompting, the model learns from limited input examples, showcasing how it can adapt to specific race scenarios and provide contextual recommendations.

- **Grounding**: The assistant's decisions are grounded in real race data, ensuring that predictions align with real-world scenarios and provide value to the strategist in the heat of the moment.

- **Structured Output/JSON Mode**: The assistant generates structured JSON outputs, making it easy to evaluate adn improve the predictionmodel as well as make it easy for the startegyst to ahve a look back at the decisions made after the race.


While the primary focus was on building and demonstrating the GenAI agent, future work will aim to improve model performance, particularly by incorporating additional contextual features and optimizing decision-making in varied and complex race conditions.

By reducing the cognitive load of strategists, this GenAI assistant enhances their ability to focus on higher-level decisions while leaving number-driven tasks to the AI, thus improving overall race strategy.


## Future Work

While the current GenAI-powered Pit Wall Assistant offers valuable support for Formula 1 strategists, there are several areas for improvement and expansion. The following outlines potential directions for future work:

### 1. **Enhanced Model Performance**
   - **Feature Expansion**: The current model's performance could be improved by incorporating additional race-related features such as pit stop windo, available tyres, real-time telemetry data (e.g., lap times, sector times, driver behavior) and others. Including more features would provide the model with a more comprehensive understanding of the race dynamics and improve decision-making.
   - **Alternative Models**: Exploring other machine learning models, such as deep learning-based architectures or ensemble methods, could potentially enhance the predictive power and accuracy of the assistant. This would also involve hyperparameter tuning and cross-validation to optimize the model's generalization across different race scenarios.

### 2. **Integration with Real-Time Race Data**
   - The assistant currently operates with static, pre-loaded race data. For real-world applications, integrating the assistant with live race feeds and telemetry would allow it to provide real-time decision-making assistance. This could involve real-time data ingestion from sensors, GPS, and communication systems within F1 teams.

### 3. **Context-Aware Decision Making**
   - The current model generates pit recommendations based purely on numerical inputs. Future iterations could incorporate more nuanced decision-making capabilities by integrating psychological factors and strategic insights from human race engineers. For example, AI could provide insights based on how competitors are performing and suggest strategies to counter specific drivers' behavior.

### 4. **Adaptive Strategies for Different Circuits**
   - Currently, the model is trained on a limited number of circuits. Expanding the dataset to include more tracks, particularly those with different weather conditions and layouts, will enable the assistant to adapt its recommendations to a wider range of environments. This would improve its utility across the entire F1 season and make it more versatile.

### 5. **Human-AI Collaboration**
   - Another key area of future work is improving the collaboration between human strategists and the AI. The assistant can be designed to generate multiple strategies or alternatives, allowing the strategist to select the best option based on their judgment and knowledge of the race. This human-AI synergy could lead to more effective decision-making and better outcomes.

By addressing these areas, the GenAI-powered Pit Wall Assistant can be refined into a more powerful, reliable, and adaptable tool that provides substantial support to F1 strategists, ultimately enhancing race strategies and decision-making.
